## Custom Text Classification using TensorFlow Lite

#### **Install TF Lite Model Maker**
Install the **TensorFlow Lite Model Maker** library. TF Lite Model Maker makes it easy to train models on custom dataset and reduces time to train by using Transfer Learning on pre-trained models.

In [ ]:
!pip install -q tflite-model-maker

#### **Install necessary libraries**

In [ ]:
import numpy as np
from numpy.random import RandomState
import pandas as pd
import os

from tflite_model_maker import model_spec
from tflite_model_maker import text_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.text_classifier import AverageWordVecSpec
from tflite_model_maker.text_classifier import DataLoader

import tensorflow as tf
assert tf.__version__.startswith('2')
tf.get_logger().setLevel('ERROR')

#### **Import dataset**
Import the training and test dataset and read them as CSV files using the Pandas library.

In [ ]:
df_train = pd.read_csv('train.csv', error_bad_lines=False, engine="python")

df_test = pd.read_csv('test.csv', error_bad_lines=False, engine="python")


#### **View dataset**
Check your dataset and see if it is properly imported or not.

In [ ]:
df_train.head()

In [ ]:
df_test.head()

#### **Choose a model architecture**
Choose any one model architecture of your choice and comment the rest. Each model architecture is different from the other and will yield different results. The MobileBERT model takes more time to train as its architecture is quite complex. However, feel free to play with different architectures until you find the best result.

In [ ]:
spec = model_spec.get('average_word_vec')
# spec = model_spec.get('mobilebert_classifier')
# spec = model_spec.get('bert_classifier')
# spec = AverageWordVecSpec(wordvec_dim=32)


#### **Customize the MobileBERT model hyperparameters**

**Note:** Run this cell only if you have chosen the `MobileBERT Classifier` model architecture.

The model parameters you can adjust are:

* `seq_len`: Length of the sequence to feed into the model.
* `initializer_range`: The standard deviation of the `truncated_normal_initializer` for initializing all weight matrices.
* `trainable`: Boolean that specifies whether the pre-trained layer is trainable.

The training pipeline parameters you can adjust are:

* `model_dir`: The location of the model checkpoint files. If not set, a temporary directory will be used.
* `dropout_rate`: The dropout rate.
* `learning_rate`: The initial learning rate for the Adam optimizer.
* `tpu`: TPU address to connect to.

For instance, you can set the `seq_len=256` (default is 128). This allows the model to classify longer text.

In [ ]:
# spec.seq_len = 256

#### **Load training and test data**
Load the training and test data CSV files to prepare the model training process. Make sure the `is_training` parameter for `test_data` is set to `False`.

In [ ]:
train_data = DataLoader.from_csv(
      filename='train.csv',
      text_column='sentence',
      label_column='label',
      model_spec=spec,
      is_training=True)

test_data = DataLoader.from_csv(
      filename='test.csv',
      text_column='sentence',
      label_column='label',
      model_spec=spec,
      is_training=False)

#### **Train model**
Start the model training on the train dataset. Feel free to play around with different no. of epochs until you find the ideal epoch value that gives the best results.

In [ ]:
model = text_classifier.create(train_data, model_spec=spec, epochs=100)

#### **Examine your model structure - Layers of the neural network**

In [ ]:
model.summary()

#### **Evaluate the model**
Evaluate the model accuracy on the test data and see for yourself if the model needs some tweakings such as increase in dataset or hyperparameter tuning in order to increase the accuracy.

In [ ]:
loss, acc = model.evaluate(test_data)

#### **Export TF Lite model**
The final model is exported as a TF Lite model which can be downloaded and directly deployed on your Android app.

In [ ]:
model.export(export_dir='average_word_vec')